In [9]:
# 1.45% for medicare
# Social Security tax rate of 6.2% and the Medicare tax rate of 1.45%, will be 7.65% for 2019
def estimateTax(taxBracket, income, standardDeduction = 0):
#     standardDeduction = 0
    income -= standardDeduction
    tax = 0
    keys = sorted(taxBracket.keys())
    for i, key in enumerate(keys):
        if i == len(keys) - 1:
            bracket = {
                "low": key,
                "high": float("inf"),
                "rate": taxBracket[key]
            }
        else:
            bracket = {
                "low": key,
                "high": keys[i+1] - 1,
                "rate": taxBracket[key]
            }
        if income < bracket["low"]: break
        tax += (min(income, bracket["high"]) - bracket["low"]) * (bracket["rate"]/100)
    return tax

# this is before 401k deduction
def calculateFica(income):
    return min(income, 132900) * 0.062 + income * 0.0145

def estimateUSTax(income):
    taxBracket = {
        0: 10,
        9701: 12,
        39476: 22,
        84201: 24,
        160726: 32,
        204101: 35,
        510301: 37
    }
    return estimateTax(taxBracket, income, 12200)

def estimateCaliTax(income):
    taxBracket = {
        0: 1,
        8544: 2,
        20255: 3,
        31969: 4,
        44377: 8,
        56085: 9.3,
        286492: 10.3,
        330884.00: 11.3,
        551473.00: 12.3,
        1000000.00: 13.3,
    }
    return estimateTax(taxBracket, income, 4401)

def estimateCompleteCaliTax(income):
    return estimateCaliTax(income) + estimateUSTax(income) + calculateFica(income)


In [13]:
from subprocess import run, PIPE
import json

def getSecret(name):
    file = open("/run/secrets/ejson/key", "r") 
    key = file.read()

    p = run(['ejson', 'decrypt', '--key-from-stdin', '/home/bcaldwell/secrets.ejson'], stdout=PIPE, stderr=PIPE,
        input=key, encoding='ascii')
    secrets = json.loads(p.stdout)
    return secrets[name]

In [59]:
def connectToPostgres(dbname = "ynab"):
    password = getSecret("postgres-password")

    from sqlalchemy import create_engine
    connectionString = 'postgresql://{}:{}@{}/{}'.format("selfops", password, 'postgres.postgres', dbname)
    engine = create_engine(connectionString)
    return engine

In [8]:
def connectToInflux(dbname = "ynab"):
    from influxdb import DataFrameClient
    client = DataFrameClient("influxdb.influxdb", 8086, '', '', dbname)

In [11]:
import pandas as pd

def printDictAsTable(dic):
    df = pd.DataFrame.from_dict(dic, orient='index', columns=[""])
    display(df)